<a href="https://colab.research.google.com/github/Vadimcha/DANO2/blob/main/DANO2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [43]:
data = pd.read_excel('./DB.xls') # База данных уже без выбросов
services = data.columns[10:]
data.columns

Index(['user_id', 'sex', 'age', 'region', 'bundle', 'num_sessions',
       'avg_session', 'service', 'order_date', 'price', 'fuel', 'cinema',
       'vkusvill', 'travel', 'restaurants', 'concerts', 'games', 'books',
       'theater', 'sports', 'beauty', 'flowers', 'technology'],
      dtype='object')

In [71]:
# Факторы объединения значений
factors = {
    'user_id': [],
    'num_sessions': ['sum'],
    'avg_session': ['mean'],
    'price': ['sum'],
    'fuel': ['sum'], 
    'cinema': ['sum'],
    'vkusvill': ['sum'], 
    'travel': ['sum'], 
    'restaurants': ['sum'], 
    'concerts': ['sum'], 
    'games': ['sum'], 
    'books': ['sum'],
    'theater': ['sum'], 
    'sports': ['sum'], 
    'beauty': ['sum'], 
    'flowers': ['sum'], 
    'technology': ['sum']
}

dataUnique = data.groupby('user_id').agg(factors).round(2)
dataUnique = dataUnique.reset_index()

dataUnique.columns = ['user_id', 'summSessions', 'avg_session', 'summPrice', 'fuel', 'cinema', 'vkusvill',	
                     'travel',	'restaurants',	'concerts',	'games',	'books',	'theater',	'sports',	
                      'beauty',	'flowers',	'technology']
# dataUnique.head(100)

In [47]:
print(dataUnique.shape[0])
print(len(dataUnique[dataUnique['fuel'] > 1]) + len(dataUnique[dataUnique['fuel'] < 2]))

9333
9333


In [100]:
# Подсчет коэффициэнта возвращаемости по формуле X/(X+Y), где X - кол-во людей кто купил категорию 2 и более раз, Y - менее соответственно

# m['X'] = [Y, Z]  значит, что в категории X, Y человек, которые покупали более одного раза, и Z человек менее
m = {}
koef = {}
for serv in services:
  m[serv] = [len(dataUnique[dataUnique[serv] >= 2]), len(dataUnique[dataUnique[serv] < 2])]
  koef[serv] = m[serv][0] / (m[serv][0] + m[serv][1])

koef = list(sorted(koef.items(), key=lambda item: item[1]))[::-1]
koef

[('cinema', 0.2256509161041466),
 ('travel', 0.0692167577413479),
 ('fuel', 0.047144540876459876),
 ('concerts', 0.015429122468659595),
 ('games', 0.012643308689596056),
 ('vkusvill', 0.010714668381013608),
 ('theater', 0.003857280617164899),
 ('books', 0.0028929604628736743),
 ('sports', 0.0022500803600128574),
 ('flowers', 0.0017143469409621772),
 ('restaurants', 0.0017143469409621772),
 ('technology', 0.0),
 ('beauty', 0.0)]

In [114]:
Users = {}  # по ключу user_id доступны все записи отсортированные по датам

for user in dataUnique['user_id']:
    Users[user] = data[data['user_id'] == user].copy()  # Создание копии DataFrame
    Users[user]['order_date'] = pd.to_datetime(Users[user]['order_date'])  # Преобразование столбца 'order_date' в формат даты
    Users[user].sort_values(by='order_date', inplace=True)

In [115]:
Users[11]

,user_id,sex,age,region,bundle,num_sessions,avg_session,service,order_date,price,...,travel,restaurants,concerts,games,books,theater,sports,beauty,flowers,technology
3,11,M,17,Нижегородская область,Tinkoff Pro,2422.0,66.21814,Топливо,2022-08-20,234.63,...,0,0,0,0,0,0,0,0,0,0
1,11,M,17,Нижегородская область,Tinkoff Premium,2422.0,66.21814,Топливо,2022-11-04,234.60,...,0,0,0,0,0,0,0,0,0,0
2,11,M,17,Нижегородская область,No bundle,2422.0,66.21814,Топливо,2022-11-21,234.63,...,0,0,0,0,0,0,0,0,0,0


In [117]:
RESULTS = {}
for serv1 in range(5):
  popular = koef[serv1][0]
  for serv in services:
    if serv == popular:
      continue
    use = 0
    not_use = 0
    for useri in dataUnique['user_id']:
      user = Users[useri]
      min_popular_index = user[user[popular] == 1].index.min()
      max_need_index = user[user[serv] == 1].index.max()
      if min_popular_index < max_need_index:
        use += 1
      else:
        not_use += 1
  RESULTS[popular].append({ serv: (use / (use + not_use)) })
RESULTS    

KeyError: ignored